# Programming Binary Logistic Regression
You can access videos of the derivation of Logistic Regression here: https://youtu.be/FGnoHdjFrJ8.


In [ ]:
# or play it in the browser right here! 
from IPython.display import YouTubeVideo
YouTubeVideo("FGnoHdjFrJ8")

___
From the videos, the main result is the *update equation* for **binary logistic regression**. The update equation for logistic regression can be written as:

$$ \underbrace{w_j}_{\text{new value}} \leftarrow \underbrace{w_j}_{\text{old value}} + \eta \underbrace{\sum_{i=1}^M (y^{(i)}-g(x^{(i)}))x^{(i)}_j}_{\text{gradient}} $$

where $\eta$ is the learning rate, $g(\cdot)$ is the sigmoid function, $y^{(i)}$ is an instance of the target, $\hat{y}^{(i)}$ is an instance of the prediction (for the current value of $w$), and $x^{(i)}_j$ is the $i^{th}$ instance of the input with the $j^{th}$ feature (that is, the feature in row $i$ and column $j$). The process is iterative, so we must calculate the gradient every time $w$ is updated. We derived this for the simple gradient ascent approach, where we calculate the gradient and step in the direction of steepest ascent. 

We can simplify this calculation a bit using matrix algebra to solve for the entire vector $w$:

$$ w \leftarrow w + \eta \sum_{i=1}^M (y^{(i)}-g(x^{(i)}))x^{(i)} $$

where we simply multiply the entire $x^{(i)}$ instance by the scalar difference, $y^{(i)}-\hat{y}^{(i)}$. 

___



Now lets start to program logistic regression in the binary case. Recall that the logistic regression equation directly regresses the logistic function (also called a sigmoid) for the probability $y=1$. In this case,

$$ p(y^{(i)}=1\text{ | }x^{(i)},w)=\frac{1}{1+\exp{(w^T x^{(i)}})}$$

Recall also that the dot product of $w^T x^{(i)}$ results in a scalar value. That is, we multiply $w$ and $x^{(i)}$ together to get a single value, then place that single value through the sigmoid to get the probability that $y=1$. The prediction can be written as:

$$ \hat{y}^{(i)} = 1 \text{      if,  } p(y^{(i)}=1\text{ | }x^{(i)},w) > 0.5  $$ and $$ \hat{y}^{(i)} = 0 \text{      otherwise  } $$

___
Let's start by programming the sigmoid and predictions. 

In [ ]:
import numpy as np
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
        
blr = BinaryLogisticRegressionBase(0.1)
print(blr)


___

Now let's add the model training function, using the update equation defined previously:

$$ w \leftarrow w + \eta \sum_{i=1}^M (y^{(i)}-g(x^{(i)}))x^{(i)} $$

In [ ]:
# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi # the actual update inside of sum
            gradient += gradi.reshape(self.w_.shape) # reshape to be column vector and add to gradient
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

            
blr = BinaryLogisticRegression(0.1)
print(blr)

To understand how our classifier works, we need some data. Let's first lets load in some example data, the iris dataset.

Notice that we have made the iris classification problem **binary**. Instead of three classes, we have made it only two classes. Most values are zero (as seen in the pie chart above). While only about a third of all instances are of class type "1".

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
import plotly

ds = load_iris()
X = ds.data
y = (ds.target>1).astype(np.int) # make problem binary

plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'labels': np.unique(y),
          'values': np.bincount(y),
            'type': 'pie'}
fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Binary Class Distribution',
                'autosize':False,
                'width':500,
                'height':300}

plotly.offline.iplot(fig)

In [ ]:
%%time
# Now we can train the classifier
blr = BinaryLogisticRegression(0.1,10)

blr.fit(X,y)
print(blr)

___
<br>
<div class="alert alert-warning">
### Self Test Question
What part of the equation is represented by this piece of code:

    self.predict_proba(xi)
    
$$ w \leftarrow w + \eta \sum_{i=1}^M \underbrace{(\underbrace{y^{(i)}}_{\bf A}-\underbrace{g(x^{(i)})}_{\bf B})}_{\bf D}\underbrace{x^{(i)}}_{\bf C} $$
</div>
___

So how well did the training go?

In [ ]:
from sklearn.metrics import accuracy_score

yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

In [ ]:
%%time
# can we do better? Maybe more iterations?
params = dict(eta=0.1,
              iterations=500)

blr = BinaryLogisticRegression(**params)
blr.fit(X,y)
print(blr)
yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

# Extending to Vectorized Programming
Excellent! We programmed the correct update equation. But, it seems to be really slow. That's because python is not fast with `for` loops. We can speed up the process using vectorization in numpy. This is fast because numpy is written in c++ under the hood. Let's change our update equation to use only linear algebra and fewer summations (which are for loops).  

Lets start with only using $X$, instead of $x^{(i)}$. $X$ is the matrix of all instances in each row: 

$$ X=\begin{bmatrix}
        \leftarrow &  x^{(1)}      & \rightarrow  \\
        \leftarrow &  x^{(2)}      & \rightarrow  \\
        &  \vdots &\\
        \leftarrow &  x^{(M)}      & \rightarrow  \\
     \end{bmatrix}
$$

Let's also create a matrix of the differences between the target and prediction:
$$ y_{diff}=\begin{bmatrix}
        y^{(1)}-g(x^{(1)})  \\
        y^{(2)}-g(x^{(2)})  \\
        \vdots \\
        y^{(M)}-g(x^{(M)})  \\
     \end{bmatrix}
$$

What we really want to do is scale each row of $X$ by the corresponding scalar value of $y_{diff}$. Luckily for us, this is exactly how `numpy` interprets multiplication of a matrix with with a column vector that has the same number of rows. So we can simply write 

$$gradient =\sum_{i=1}^M (y^{(i)}-g(x^{(i)}))x^{(i)}$$

as

$$gradient = X*y_{diff}$$

where $*$ is a row-wise operator.

In [ ]:
%%time
# now lets do some vectorized coding
import numpy as np
from scipy.special import expit

class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

# use same params as defined above
blr = VectorBinaryLogisticRegression(**params)
blr.fit(X,y)
print(blr.w_)
yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

<br>
<div class="alert alert-warning">
### Self Test Question: 
If we had a class imbalance problem, with one type of class dominating the response, how could we manipulate the above `fit` method to give more importance to a given class?
- **A.** Add weights to the gradient calculation (for each instance of ydiff)
- **B.** Repeat certain classes in the update of the gradient calculation
- **C.** Update objective function to only look at performance on the worst performing class

</div>
___

# Programming Multiclass Logistic Regression
Our previous methods only worked for a single class, to extend this beyond binary classification we can use a common trick: train a separate Binary Classifier for each unique class. During prediction, we run an instance through each Binary Classifier, choosing the Binary Classifier that has the highest probability. **This is a method known as one-versus-all** because we only assume one class is positive when we train each separate Binary Classifier. 

This graphic breaks down the process:
![OVA](http://www.saberismywife.com/2016/11/29/Machine-Learning-3/21.png)

To program this, its surprisingly simple now that we have a fast binary classifier. 

In [ ]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row
    
lr = LogisticRegression(0.1,1500)
print(lr)

In [ ]:
%%time
ds = load_iris()
X = ds.data
y = ds.target # note problem is NOT binary anymore, there are three classes!

lr = LogisticRegression(0.1,500)
lr.fit(X,y)
print(lr)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

# Comparing to Scikit Learn
Of course, there was no **need** to program the above except for the learning experience (or because we want to extend the learning algorithm without adjusting the scikit-learn source code). Now, let's use the builtin API for `sklearn`. 

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression() # all params default

lr_sk.fit(X,y)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Why is this so much faster? The answer is parallelization is better handled through sklearn, separating out each classification, and the operations are ALL in lower level c++. **The main reason, however, is that  the update used is more advanced than what we have talked about.** They have an array of solvers that use the gradient (and Hessian) to solve the optimization more quickly. The accuracy is higher because this logistic regression object uses regularization, which helps control for over-learning, and also because the optimization allows for finer steps when we get to places where the gradient is small and the objective function is less well behaved. 
____
## Logistic Regression Class Boundaries

In [ ]:
# linear boundaries visualization from sklearn documentation
from matplotlib import pyplot as plt
import copy
%matplotlib inline

def plot_decision_boundaries(lr,Xin,y,title=''):
    Xb = copy.deepcopy(Xin)
    lr.fit(Xb[:,:2],y) # train only on two features

    h=0.01
    # create a mesh to plot in
    x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
    y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # get prediction values
    Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)

    # Plot also the training points
    plt.scatter(Xb[:, 0], Xb[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plt.show()
    
lr = LogisticRegression(0.1,1500) # this is still OUR LR implementation
plot_decision_boundaries(lr,X,y)

## Can we control these boundaries?
Let's introduce another parameter in the objective function called the L2-Norm. The idea here is that we want to keep the values of the weights relatively small. This helps to control overfitting to the data and thus . We are changing our objective function by adding in a new summation of the weights:

$$ l(w)_{reg} = l(w)_{old} + C\cdot\sum_j w_j^2 $$

This means the gradient will be updated as follows:

$$ \underbrace{w_j}_{\text{new value}} \leftarrow \underbrace{w_j}_{\text{old value}} + \eta \underbrace{\left[\left(\sum_{i=1}^M (y^{(i)}-g(x^{(i)}))x^{(i)}_j\right) + C \cdot 2w_j \right]}_{\text{new gradient}}$$

Which is the same as:
$$ w \leftarrow w + \eta \left[\underbrace{\nabla l(w)_{old}}_{\text{old gradient}} + C \cdot 2w \right]$$

Its fairly easy to implement because we just need to add to the existing gradient function.

In [ ]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        # add in regularization (to all except bias term)
        gradient[1:] += 2 * self.w_[1:] * self.C
        return gradient
        

In [ ]:
# now redefine the Logistic Regression Function where needed
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [ ]:
from ipywidgets import widgets as wd

cost_vals = np.logspace(-3,-2,15)
def lr_explor(cost_idx):
    C = cost_vals[cost_idx]
    lr_clf = RegularizedLogisticRegression(eta=0.1,
                                           iterations=2500,
                                           C=C) # get object
    
    plot_decision_boundaries(lr_clf,X,y,title="C=%.5f"%(C))

wd.interact(lr_explor,cost_idx=(0,15,1),__manual=True)

# Extended Logistic Regression Example

In this example we will explore methods of using logistic regression in scikit-learn. A basic understanding of scikit-learn is required to complete this notebook, but we start very basic. Note also that there are more efficient methods of separating testing and training data, but we will leave that for a later lecture.

First let's load a dataset and prepare it for analysis. We will use pandas to load in data, and then prepare it for classification. We will be using the titanic dataset (a very modest sized data set of about 1000 instances). The imputation methods used here are discussed in a previous notebook.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/titanic.csv') # read in the csv file

# 1. Remove attributes that just arent useful for us
del df['PassengerId']
del df['Name']
del df['Cabin']
del df['Ticket']

# 2. Impute some missing values, grouped by their Pclass and SibSp numbers
df_grouped = df.groupby(by=['Pclass','SibSp'])

# # now use this grouping to fill the data set in each group, then transform back
# fill in the numeric values
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))
# fill in the categorical values
df_imputed[['Sex','Embarked']] = df_grouped[['Sex','Embarked']].apply(lambda grp: grp.fillna(grp.mode()))
# fillin the grouped variables from original data frame
df_imputed[['Pclass','SibSp']] = df[['Pclass','SibSp']]

# 4. drop rows that still had missing values after grouped imputation
df_imputed.dropna(inplace=True)

# 5. Rearrange the columns
df_imputed = df_imputed[['Survived','Age','Sex','Parch','SibSp','Pclass','Fare','Embarked']]

df_imputed.info()

In [ ]:
# perform one-hot encoding of the categorical data "embarked"
tmp_df = pd.get_dummies(df_imputed.Embarked,prefix='Embarked')
df_imputed = pd.concat((df_imputed,tmp_df),axis=1) # add back into the dataframe

# replace the current Sex atribute with something slightly more intuitive and readable
df_imputed['IsMale'] = df_imputed.Sex=='male' 
df_imputed.IsMale = df_imputed.IsMale.astype(np.int)

# Now let's clean up the dataset
if 'Sex' in df_imputed:
    del df_imputed['Sex'] # if 'Sex' column still exists, delete it (as we created an ismale column)
    
if 'Embarked' in df_imputed:    
    del df_imputed['Embarked'] # get reid of the original category as it is now one-hot encoded
    
# Finally, let's create a new variable based on the number of family members
# traveling with the passenger

# notice that this new column did not exist before this line of code--we use the pandas 
#    syntax to add it in 
df_imputed['FamilySize'] = df_imputed.Parch + df_imputed.SibSp
df_imputed.info()

## Training and Testing Split
For training and testing purposes, let's gather the data we have and grab 80% of the instances for training and the remaining 20% for testing. Moreover, let's repeat this process of separating the testing and training data three times. We will use the hold out cross validation method built into scikit-learn.

In [ ]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'Survived' in df_imputed:
    y = df_imputed['Survived'].values # get the labels we want
    del df_imputed['Survived'] # get rid of the class label
    norm_features = ['Age','Fare' ]
    df_imputed[norm_features] = (df_imputed[norm_features]-df_imputed[norm_features].mean()) / df_imputed[norm_features].std()
    X = df_imputed.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(
                         n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

In [ ]:
# run logistic regression and vary some parameters
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = RegularizedLogisticRegression(eta=0.1,iterations=2000) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1
    
# Also note that every time you run the above code
#   it randomly creates a new training and testing set, 
#   so accuracy will be different each time

In [ ]:
# this does the exact same thing as the above block of code, but with shorter syntax

for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    lr_clf.fit(X[train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict(X[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

In [ ]:
from ipywidgets import widgets as wd

num_cv_iterations = 10
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.5)

def lr_explor(cost):
    print('Running')
    lr_clf = RegularizedBinaryLogisticRegression(eta=0.05,
                                                 iterations=1000,
                                                 C=float(cost)) # get object
    acc = []
    for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
        lr_clf.fit(X[train_indices],y[train_indices])  # train object
        y_hat = lr_clf.predict(X[test_indices]) # get test set predictions
        acc.append(mt.accuracy_score(y[test_indices],y_hat))
        
    acc = np.array(acc)
    print(acc.mean(),'+-',2*acc.std())
        
wd.interact(lr_explor,cost=list(np.logspace(-4,1,15)),__manual=True)